In [1]:
# -----------------------------
# MNIST DIGIT CLASSIFIER (PyTorch)
# -----------------------------

import torch
import gradio as gr
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# -----------------------------
# 1. LOAD DATA
# Transforms are preprocessing steps that get applied automatically to every image
# you load from a dataset. 
# Think of transforms as a recipe that says:

# "Every time you give me an image, do X, then Y, then Z to it."
# "For every MNIST image: convert it to a PyTorch tensor.
# MNIST images come in as PIL images (Python Imaging Library).

# But your neural network expects tensors.
# -----------------------------
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load training dataset (MNIST)
train_dataset = datasets.MNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True
)

# Load test dataset
test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True
)

# Make DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Access and print the unique labels in the training data set
print("Unique labels in training dataset:", train_dataset.classes)
print("Label range:", torch.unique(train_dataset.targets))

Unique labels in training dataset: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
Label range: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [3]:
# -----------------------------
# 2. DEFINE NEURAL NETWORK
# Design a Neural Network with 1 hidden layer of 128 neurons
# -----------------------------
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Define layers
        self.fc1 = nn.Linear(28*28, 128)  # Input layer: 784 → 128
        self.fc2 = nn.Linear(128, 10)     # Output layer: 128 → 10 digits
        
    def forward(self, x):
        # Flatten image: (batch, 1, 28, 28) → (batch, 784)
        x = x.view(-1, 28*28)

        # Add activation between layers
        x = torch.relu(self.fc1(x))  # Hidden layer with ReLU activation

        # Output layer
        x = self.fc2(x)  # Raw logits (no activation)
        
        return x

# Create the model
model = SimpleNN()
print(f"\nModel architecture:\n{model}")


Model architecture:
SimpleNN(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [4]:
# 3. LOSS FUNCTION + OPTIMIZER
# -----------------------------
# Define loss function
criterion = nn.CrossEntropyLoss()

# Setup gradient descent with learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
# -----------------------------
# 4. TRAINING LOOP
# -----------------------------

# Define the number of epochs
epochs = 5

print(f"\nStarting training for {epochs} epochs...\n")

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        # Reset the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagate
        loss.backward()
        
        # Update gradients
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


Starting training for 5 epochs...

Epoch 1/5, Loss: 1169.6112
Epoch 2/5, Loss: 463.5245
Epoch 3/5, Loss: 368.0057
Epoch 4/5, Loss: 329.8093
Epoch 5/5, Loss: 306.6636


In [6]:
# -----------------------------
# 5. EVALUATION
# -----------------------------
print("\nEvaluating on test set...")

correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        # Forward pass
        outputs = model(images)

        # Predicted class = index of max logit
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Evaluating on test set...
Test Accuracy: 91.64%


In [7]:
# -----------------------------
# 6. TEST SINGLE PREDICTION
# -----------------------------
# ------------------------------
# Gradio Sketchpad gives you:

# * a full-color NumPy array

# * black digit on white background

# * large resolution

# * no consistent scale
#
# Hence the preprocessing
# ------------------------------

def preprocess_image(image):
    sketch_transform = transforms.Compose([
    transforms.ToPILImage(),                      # NumPy → PIL
    transforms.Grayscale(),                       # ensure 1 channel
    transforms.Resize((28, 28)),                  # 28x28 like MNIST
    transforms.Lambda(lambda img: ImageOps.invert(img)),  # invert colors
    transforms.ToTensor(),                        # → tensor, shape (1,28,28), values in [0,1]
    ])
    # Gradio Sketchpad sometimes passes a dict with 'composite'
    if isinstance(image, dict):
        image = image['composite']   # this is a NumPy array
    
    # Apply the preprocessing transform
    img_tensor = sketch_transform(image)  # (1, 28, 28)
    
    # Add batch dimension → (1, 1, 28, 28)
    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor

def predict_digit(image):
    # --- STEP 1: CHECK IF SOMETHING HAS BEEN DRAWN ---
    if image is None: return "Draw something!"

    # --- STEP 2: PREPROCESS THE IMAGE ---
    img_tensor = preprocess_image(image)
    
    # --- STEP 3: RUN THE MODEL ---
    model.eval()
    with torch.no_grad():
        prediction = model(img_tensor)
        
        # Get the index of the highest score (the predicted digit)
        predicted_digit = torch.argmax(prediction).item()
        
    return str(predicted_digit)

# UI Setup
print("\nLaunching Gradio interface...")
interface = gr.Interface(
    fn=predict_digit, 
    inputs=gr.Sketchpad(label="Draw a digit (0-9)"), 
    outputs=gr.Label(label="Predicted Digit"),
    title="MNIST Digit Classifier",
    description="Draw a digit and the neural network will predict what number it is!"
)
interface.queue().launch()


Launching Gradio interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
